In [1]:
import numpy as np
import pandas as pd
import scipy.constants as sc
from pygmid import Lookup as lk

#### Technology data

In [2]:
n = lk('../../techsweep/simulation/nfet_01v8_lvt.mat')
p = lk('../../techsweep/simulation/pfet_01v8.mat')

#### Baseline sizing

In [3]:
T = 300; ib = 10e-6; cl = 1e-12
type = ['n', 'n', 'p']
l = np.array([1.0, 0.5,  1.0])
w = np.array([2.0, 20.0, 2.0])
nf = np.array([2, 4, 2])

#### Estimated operating point parameters

In [4]:
id = np.array([ib, ib/2, ib/2])
gm_id=np.zeros(len(l))
gm_cgg=np.zeros(len(l))
gm_cdd=np.zeros(len(l))
gm_cgd=np.zeros(len(l))
gm_gds=np.zeros(len(l))
sth_gm=np.zeros(len(l))

for i in range(len(l)):
    if type[i] == 'n':
      gm_id[i] = n.lookup('GM_ID', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_cgg[i] = n.lookup('GM_CGG', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_cdd[i] = n.lookup('GM_CDD', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_cgd[i] = n.lookup('GM_CGD', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_gds[i] = n.lookup('GM_GDS', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      sth_gm[i] = n.lookup('STH_GM', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
    else:
      gm_id[i] = p.lookup('GM_ID', ID_W=id[i]/w[i], L=l[i]) 
      gm_cgg[i] = p.lookup('GM_CGG', ID_W=id[i]/w[i], L=l[i])
      gm_cdd[i] = p.lookup('GM_CDD', ID_W=id[i]/w[i], L=l[i])
      gm_cgd[i] = p.lookup('GM_CGD', ID_W=id[i]/w[i], L=l[i])
      gm_gds[i] = p.lookup('GM_GDS', ID_W=id[i]/w[i], L=l[i])
      sth_gm[i] = p.lookup('STH_GM', ID_W=id[i]/w[i], L=l[i])

gm = id*gm_id
gds = gm/gm_gds
cgg = gm/gm_cgg
cdd = gm/gm_cgg
cgd = gm/gm_cgd
gamma = sth_gm/(4*sc.Boltzmann*T)

df = pd.DataFrame( [gm_id, gm_cgg/1e9/2/np.pi, gm_gds, gm/1e-6, gds/1e-6, cgg/1e-15, cdd/1e-15, cgd/1e-15, gamma], \
                   ['gm_id (S/A)', 'ft (GHz)', 'gm_gds', 'gm (uS)', 'gds (uS)', 'cgg (fF)', 'cdd (fF)', 'cgd (fF)', 'gamma'], columns=['M0', 'M1', 'M2']); df.round(2)


,M0,M1,M2
gm_id (S/A),9.42,22.98,7.18
ft (GHz),1.27,0.36,0.52
gm_gds,34.56,57.54,253.37
gm (uS),94.21,114.91,35.90
gds (uS),2.73,2.00,0.14
cgg (fF),11.83,51.24,10.90
cdd (fF),11.83,51.24,10.90
cgd (fF),0.81,5.00,0.11
gamma,1.86,1.30,1.15


#### Estimate performance specs

In [5]:
# A0
A0 = gm[1]/(gds[1]+gds[2])

# UGF
cltot = cl + cdd[1] + cdd[2]
UGF = gm[1]/cltot/2/np.pi

# PM
fp2 = gm_cgg[2]/2/np.pi
phip2 = -np.arctan(UGF/fp2)*180/np.pi
fz2 = 2*fp2
phiz2 = +np.arctan(UGF/fz2)*180/np.pi
fz3 = gm[1]/cgd[1]/2/np.pi
phiz3 = -np.arctan(UGF/fz3)*180/np.pi
PM= 90 +phip2 +phiz2 +phiz3 

# NOI
NOI = ( gamma[1]*(1+2*gamma[2]/gamma[1]) * sc.Boltzmann*T/cltot)**0.5

df = pd.DataFrame( [A0, UGF/1e6, PM, NOI/1e-6], \
                   ['A0', 'UGF (MHz)', 'PM (deg)', 'Noise (uVrms)'], columns=['Value']); df.round(2)

,Value
A0,53.73
UGF (MHz),17.22
PM (deg),88.79
Noise (uVrms),118.46


In [6]:
df = pd.DataFrame( [fp2/1e6, fz2/1e6, fz3/1e6, phip2, phiz2, phiz3], \
                   ['fp2 (MHz)', 'fz2 (MHz)', 'fz3 (MHz)', 'phip2 (deg)', 'phiz2 (deg)', 'phiz2 (deg)'], columns=['Value']); df.round(2)

,Value
fp2 (MHz),524.04
fz2 (MHz),1048.08
fz3 (MHz),3654.91
phip2 (deg),-1.88
phiz2 (deg),0.94
phiz2 (deg),-0.27
